<a href="https://colab.research.google.com/github/wiendewani/Image-Retrieval/blob/master/Final%20Code%20/%20VGG19_Image_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import numpy as np
import os
import json
import cv2
import sklearn.metrics as metric
from tensorflow.keras.models import Model
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def euclidean(a, b):
	return np.linalg.norm(a - b)

In [ ]:
def perform_search(query_features, indexed_train, max_results=5):
	retrieved = []
	for idx in range(0, len(indexed_train["features"])):
		distance = euclidean(query_features, indexed_train["features"][idx])
		retrieved.append((distance, idx))
	retrieved = sorted(retrieved)[:max_results]
	return retrieved

In [ ]:
base_dataset = "/content/drive/MyDrive/Kembali Citra Notebook/dataset"
class_dir = ['beaches', 'bus', 'dinosaurs',
             'elephants', 'flowers', 'foods',
             'horses', 'monuments', 'mountains_and_snow','peolpe_and_villages_in_Africa'
             ]

IMAGE_SIZE = (224, 224)

In [ ]:
# indexing file images
dataset_test = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'test_set', class_item)
    for file in os.listdir(cur_dir):
        dataset_test.append(os.path.join(cur_dir, file))

In [ ]:
print("len to retrieving:", len(dataset_test))

len to retrieving: 100


In [ ]:
print("[INFO] load images Corel-Image 1k dataset...")
#  load images
test_images = []
for image_path in dataset_test:
    if ".jpg" or ".jpeg" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        test_images.append(image)

[INFO] load images Corel-Image 1k dataset...


In [ ]:
print("[INFO] normalization...")
test_x = np.array(test_images).astype("float32") / 255.0

[INFO] normalization...


In [ ]:
from tensorflow.keras.applications import VGG19
modelVGG19 = VGG19(weights="imagenet")
model = Model(inputs=modelVGG19.input, outputs=modelVGG19.get_layer("fc1").output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
features = model.predict(test_x)

In [ ]:
with open('/content/drive/MyDrive/Kembali Citra Notebook/Notebook/feature_extraction2.json') as f:
  training_indexed = json.load(f)

In [ ]:
query_indexes = list(range(0, test_x.shape[0]))
label_builder = list(np.unique(training_indexed["labels"]))
class_builder = {label_unique:[] for label_unique in label_builder}
recalls = copy.deepcopy(class_builder)
precisions = copy.deepcopy(class_builder)
# loop over the testing indexes
for i in query_indexes:
    queryFeatures = features[i]
    results = perform_search(queryFeatures, training_indexed, max_results=5)
    labels_ret = [training_indexed["labels"][r[1]] for r in results]
    label_true = dataset_test[i].split("/")[7]
    label_trues = [label_true for _ in labels_ret]
    recall = metric.recall_score(label_trues, labels_ret, average='weighted')
    precision = metric.precision_score(label_trues, labels_ret, average='weighted')
    recalls[label_true].append(recall)
    precisions[label_true].append(precision)

In [ ]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))

recall values:
beaches 0.54
bus 1.0
dinosaurs 1.0
elephants 0.74
flowers 1.0
foods 0.48
horses 0.9199999999999999
monuments 0.7000000000000001
mountains_and_snow 0.76
peolpe_and_villages_in_Africa 0.6799999999999999
combined recall 0.782

precision values:
beaches 0.8
bus 1.0
dinosaurs 1.0
elephants 1.0
flowers 1.0
foods 0.8
horses 1.0
monuments 0.9
mountains_and_snow 1.0
peolpe_and_villages_in_Africa 0.9
combined precision 0.9400000000000001


In [ ]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))

recall values:
beaches 0.54
bus 1.0
dinosaurs 1.0
elephants 0.74
flowers 1.0
foods 0.48
horses 0.9199999999999999
monuments 0.7000000000000001
mountains_and_snow 0.76
peolpe_and_villages_in_Africa 0.6799999999999999
combined recall 0.782

precision values:
beaches 0.8
bus 1.0
dinosaurs 1.0
elephants 1.0
flowers 1.0
foods 0.8
horses 1.0
monuments 0.9
mountains_and_snow 1.0
peolpe_and_villages_in_Africa 0.9
combined precision 0.9400000000000001
